In [1]:
import os
from datetime import datetime
import pandas as pd
import glob

import pyarrow as pa
import pyarrow.parquet as pq

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
cwd = os.getcwd()
print("Current Working Directory:", cwd)

Current Working Directory: c:\Users\35385\Desktop\CS_Summer_2024\Shared_GH\New-York-App\data-analytics\analysis


In [3]:
combined_df_dir = os.path.join(os.getcwd(), "..", "Datasets", "taxi_other")

# Define the directory where the data is located relative to the current working directory
print("Taxi Zone CSV Directory:", combined_df_dir)

# Define the file path relative to the data directory
combined_df_path = os.path.join(cwd, combined_df_dir, "combined_df.csv")

combined_df = pd.read_csv(combined_df_path, keep_default_na=True, delimiter=",", skipinitialspace=True, encoding="Windows-1252")

Taxi Zone CSV Directory: c:\Users\35385\Desktop\CS_Summer_2024\Shared_GH\New-York-App\data-analytics\analysis\..\Datasets\taxi_other


In [4]:
combined_df

,datetime_formatted,hour,day_of_week,week,month,day_of_month,year_month,zone,passenger_count
0,2021-01-01 00:00:00,0,4,53,0,1,2021-01,4,4
1,2021-01-01 00:00:00,0,4,53,0,1,2021-01,13,3
2,2021-01-01 00:00:00,0,4,53,0,1,2021-01,17,1
3,2021-01-01 00:00:00,0,4,53,0,1,2021-01,24,3
4,2021-01-01 00:00:00,0,4,53,0,1,2021-01,33,2
...,...,...,...,...,...,...,...,...,...
3343047,2024-04-01 00:00:00,0,0,14,3,1,2024-04,261,11
3343048,2024-04-01 00:00:00,0,0,14,3,1,2024-04,262,35
3343049,2024-04-01 00:00:00,0,0,14,3,1,2024-04,263,56
3343050,2024-04-01 00:00:00,0,0,14,3,1,2024-04,264,8


In [ ]:
X = zone_busy_df[['hour', 'day_of_week', 'week', 'month', 'zone']]
y = zone_busy_df['passenger_count']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['hour', 'week']),
        ('cat', OneHotEncoder(), ['day_of_week', 'month', 'zone'])
    ])

# Example model pipeline (using a simple linear regression model)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the model
model.fit(X_train, y_train)

# Evaluate the model
score = model.score(X_test, y_test)
print(f"Model R^2 Score: {score}")

# Predictions (for example purposes)
predictions = model.predict(X_test)